<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# ***A influência das opções no preço do tivo subjacente.***
---

$\rightarrow$ **esta célula deve ser removida na entrega do projeto**

# Objetivo

Neste trabalho você deve obrigatoriamente trabalhar em grupo de 2-4 alunos e desenvolver uma solução de aprendizado supervisionado para algum problema de interesse do grupo. Os problemas poderão ser de classificação ou de regressão.

## Descrição do Projeto

O trabalho deve ser original e desenvolvido em R ou Python notebook com um conjunto de dados próprios e originais definidos. Entendam como dados próprios e originais dados que são originais de empresas ou entidades, podendo ser bases públicas. Dados de ONGs, empresas privadas ou governo, são exemplos de bases válidas. Bases de repositórios públicos como Kaggle e UCI, que fornecem bases prontas não são originais. Tenha em mente: você deve construir uma base de dados nova para o seu problema, é essa a ideia de dados originais. Se tiver dúvidas consulte o professor antes de empregar uma base.

Os dados devem suportar um problema relevante e adequado para o aprendizado supervisionado. Assim, não podem ser empregadas bases artificiais e problemas muito simples. Devem ser evitadas também Séries Temporais como séries financeiras (como ações) e de dados climáticos que requerem outras técnicas não detalhadas aqui.

As bases devem conter um mínimo de 1000 instâncias. Consulte o professor para uso de um número menor de instâncias para dados próprios de uma empresa por exemplo.

Você deve implementar ao menos dois modelos, comparar e discutir os seus resultados. Seu notebook deve ser 100% executável e incluir todo necessário para construção dos dados, entendimento do problema e da solução.

Deve ainda ser entregue uma apresentação em **vídeo (até 5min)** dos resultados.



$\rightarrow$ **esta célula deve ser removida na entrega do projeto**

Siga este template para entrega do seu projeto. O código deste notebook precisa ser 100% executável, sem a necessidade de qualquer operação adicional como por exemplo a download, cópia ou alteração de arquivos. Crie assim repositórios públicos para suas bases de dados, por exemplo, disponibilizando-as no GitHub.

## Entregas na Tarefa do Moodle

1. Poste este Python notebook (.ipynb) modificado com o seu projeto.
2. Poste o endereço (link) do Python notebook do seu projeto compartilhado (público para leitura) nos comentários da tarefa
3. Um vídeo de até 5min com a apresentação do seu projeto em formato .mp4
4. Opcional, poste o .pptx da apresentação

## Vídeo

O vídeo deve contemplar:

1. Apresentação e Justificativa do seu problema
2. A abordagem dada ao problema (fonte dos dados, transformações, modelos avaliados)
3. A solução e seus resultados
4. Diferenciais e Melhorias Futuras do trabalho (modelos e técnicas diferenciadas que foram empregadas, limitações do modelo atual e como melhora-las etc.)







In [ ]:
#@title Identificação do Grupo

#@markdown Integrantes do Grupo (*informe \<TIA\>,\<nome\>*)
Aluno1 = '10423914, Humberto Guttau Bravo' #@param {type:"string"}
Aluno2 = '1043651, Ricardo Zulian de Souza Amaral' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



# **Apresentação**

# Problema

*Apresente resumidamente o trabalho justificando a escolha e a relevância do problema.*



# Referencial Teórico

*Apresente aqui o referencial teórico empregado. Empregue as referências empregadas que devem aparecer no final do trabalho. Lembre-se, existem  referências técnicas do domínio da solução, mas também referências do domínio do problema (por exemplo, se estiver classificando reclamações de clientes, existem inúmeras referências sobre o problema de negócio em si).*

# Metodologia

*Apresente e discuta a abordagem de solução empregada. Discuta os dados empregados, tratamento efetuados, variáveis preditoras e objetivo, técnicas empregadas.*



# Resultados

*Resuma aqui os resultados, principalmente incluindo tabelas e esquemas necessários.*

# **Implementação**

# Base de Dados

A base de dados tem origem nos data-dumps diários da B3. Existe um sistema próprio que faz o download diário dos arquivos texto gerados pela bolsa e os carrega em uma base de dados ms-sql.

A característica formal da base já garante consistência, como pode ser visto na ausência de nulos na base bruta.




In [23]:
# seu código
import pandas as pd
# Carregando base de preços de fechamento de PETR4
df_p4 = pd.read_csv('petr4.csv.gz', compression='gzip')
df_p4_opc = pd.read_csv('petr4_opc.csv.gz', compression='gzip')
print("Base do Ativo:")
print(df_p4.info())
print(df_p4.sample(5))
print("\nBase do Derivativo:")
print(df_p4_opc.info())
print(df_p4_opc.sample(5))

print('\nBase do ativo entre: ', df_p4['tradedate'].min(), ' e ', df_p4['tradedate'].max())
print('\nBase do derivativo entre: ', df_p4_opc['tradedate'].min(), ' e ', df_p4_opc['tradedate'].max())

Base do Ativo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tradedate   643 non-null    object 
 1   ticker      643 non-null    object 
 2   price_last  643 non-null    float64
dtypes: float64(1), object(2)
memory usage: 15.2+ KB
None
      tradedate        ticker  price_last
544  2025-03-11  PETR4              34.10
436  2024-09-30  PETR4              36.01
316  2024-04-11  PETR4              39.30
266  2024-01-29  PETR4              40.57
218  2023-11-16  PETR4              35.55

Base do Derivativo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604847 entries, 0 to 604846
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   asset_id        604847 non-null  int64  
 1   tradedate       604847 non-null  object 
 2   ticker          604847 non-null  object 
 3  

Existem dois tipos de opções: opções de compra 'CALL' e opções de venda 'PUT'. os comportamentos de preço são invertidos e é necessário separa-las para processamento. Trbalharemos aqui apenas com opções de compra. A coluna com esta definição será eliminada.

In [21]:
print(len(df_p4_opc), 'registros totais.')
df_p4_opc = df_p4_opc[df_p4_opc['segment'] == 'EQUITY CALL']
df_p4_opc = df_p4_opc.drop(columns=['segment'])
print(len(df_p4_opc), 'registros de calls após a limpeza.')
print(df_p4_opc.sample(5))

604847 registros totais.
['asset_id', 'tradedate', 'ticker', 'asset_ticker', 'segment', 'strike', 'style', 'pos_covered', 'pos_blocked', 'pos_uncovered', 'oi', 'lenders', 'borrowers', 'iq', 'timo_to_expiry', 'moneyness', 'vol_implied']
314083 registros de calls após a limpeza.
        asset_id   tradedate    ticker asset_ticker  strike style  \
340060     12110  2024-03-18   PETRE28        PETR4   28.75     A   
485343     12722  2024-08-12  PETRA419        PETR4   38.14     E   
186363      7405  2023-09-14  PETRK215        PETR4   20.45     E   
148166      6209  2023-07-26  PETRH243        PETR4   16.45     A   
180255      7380  2023-09-06  PETRK322        PETR4   32.20     A   

        pos_covered  pos_blocked  pos_uncovered     oi  lenders  borrowers  \
340060            0            0           7100   7100        2          1   
485343            0         3800              0   3800        3          1   
186363        13000            0           2700  15700        3          

O rótulo de classificação será definido como se cada contrato se comportou como uma 'barreira' no pregão seguinte: o preço do ativo não conseguiu romper o strike da opção. Para isso precisamos deslocar os preços de fechamento do ativo para incorporação na base de derivativos.

In [13]:
#Criando a coluna 'next_price_last' contendo o fechamento do dia seguinte do ativo.

df_p4['next_price_last'] = df_p4['price_last'].shift(-1).fillna(0)
print(df_p4.head())

    tradedate        ticker  price_last  next_price_last
0  2023-01-02  PETR4              22.92            22.34
1  2023-01-03  PETR4              22.34            23.05
2  2023-01-04  PETR4              23.05            23.88
3  2023-01-05  PETR4              23.88            23.74
4  2023-01-06  PETR4              23.74            23.87


In [14]:
#incorporando a coluna next_price_last no dataframe de opções
df_p4_opc = pd.merge(df_p4_opc, df_p4[['tradedate', 'next_price_last']], on='tradedate', how='left')
print(df_p4_opc.head())

   asset_id   tradedate    ticker asset_ticker      segment  strike style  \
0      1231  2023-01-02    PETRA1        PETR4  EQUITY CALL   12.51     A   
1      1231  2023-01-02  PETRA108        PETR4  EQUITY CALL   13.76     E   
2      1231  2023-01-02  PETRA130        PETR4  EQUITY CALL   13.01     A   
3      1231  2023-01-02  PETRA132        PETR4  EQUITY CALL   13.26     E   
4      1231  2023-01-02  PETRA140        PETR4  EQUITY CALL    1.39     A   

   pos_covered  pos_blocked  pos_uncovered      oi  lenders  borrowers  \
0        29700        10000         127100  166800       15         28   
1         1900        26900          11200   40000        4         11   
2         3600        18100          13600   35300        3          7   
3        34400        26300         135100  195800        6         26   
4            0        60003           1251   61254       14          5   

         iq  timo_to_expiry  moneyness  vol_implied  next_price_last  
0  0.535714        0.

## **Modelo 1**

*Descreva resumidamente o modelo empregado.*


### **Modelo 1:** Preparação dos Dados


In [ ]:
# seu código

### **Modelo 1:** Modelo


In [ ]:
# seu código

### **Modelo 1:** Resultados


In [ ]:
# seu código

## **Modelo 2**

*Descreva resumidamente o modelo empregado.*


### **Modelo 2:** Preparação dos Dados


In [ ]:
# seu código

### **Modelo 2:** Modelo


In [ ]:
# seu código

### **Modelo 2:** Resultados


In [ ]:
# seu código

# **Conclusão**

*Apresente a conclusão do seu estudo comparando ainda os resultados obtidos com o referencial teórico apresentado.*



# **Referências**

*Indique as referências empregadas, incluindo as fontes de dados.*

---

In [ ]:
#@title Avaliação
Completo = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Projeto cumpre todos os itens pedidos.
Relevancia = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As seleções de dados e eventos para análise são relevantes e justificados.
Tecnicas = 6 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As técnicas de empregadas são adequadas e corretamente aplicadas.
Apresentacao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown A apresentação dos resultados é clara e objetiva.
Analise = 8 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As premissas de análise se justificam e a analise é correta.
Conclusao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As conclusões são justificadas e relevantes
Bonus = 0.5 #@param {type:"slider", min:0, max:1, step:0.5}
#@markdown A critério do professor por inovações na abordagem e no uso de técnicas de Análise de Dados








In [ ]:
#@markdown ### Nota Final
nota = Completo + Relevancia + Tecnicas + Apresentacao + Analise + Conclusao

nota = nota / 6 + Bonus

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
alunos

Nota final do trabalho 8.3



,tia,nome,nota
0,1115665,ADRIANA FUJITA,8.3
1,1115677,DANIEL HENRIQUE,8.3
